# SPLADE

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-8"

## Libraries Installation

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.7 MB/s eta 0:00:00


In [ ]:
!pip install trectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for trectools: filename=trectools-0.0.49-py3-none-any.whl size=27141 sha256=fe346323ae67c1210fb9e569fa536929a1f88ee9be50f66318f3f607745fb029
  Stored in directory: /root/.cache/pip/wheels/b2/1d/4d/445b0fb9a145de0dc24861a535cbe755f637327da7f5d65ed7
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=af96835b902469237cb641042c7ccdeaf3968aa523c17e56ed821e1895086ef1
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built trectools bs4


In [ ]:
import torch
import jsonlines
import pickle

## Pretrained model and tokenizer

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, DistilBertTokenizer, DistilBertForMaskedLM
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
model_name_1 = 'naver/splade_v2_distil' 
model_name_2 = 'naver/splade-cocondenser-selfdistil'
model_name_3 = 'naver/splade-cocondenser-ensembledistil' 

tokenizer_1 = DistilBertTokenizer.from_pretrained(model_name_1)
model_1 = DistilBertForMaskedLM.from_pretrained(model_name_1)

tokenizer_2 = BertTokenizer.from_pretrained(model_name_2)
model_2 = BertForMaskedLM.from_pretrained(model_name_2)

tokenizer_3 = AutoTokenizer.from_pretrained(model_name_3)
model_3 = AutoModelForMaskedLM.from_pretrained(model_name_3)

## Dataset definition

In [ ]:
model = model_3
tokenizer = tokenizer_3

In [ ]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts['input_ids'])
    
    def __getitem__(self, idx):
        #return self.tokenized_texts[idx]
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'special_tokens_mask': self.tokenized_texts['special_tokens_mask'][idx]
        }

In [ ]:
from transformers import BatchEncoding

def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

## Sparse Vectorization

In [ ]:
max_length = 256

In [ ]:
from torch.nn.functional import relu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def vectorize_to_sparse(text, tokenizer=tokenizer_1, model=model_1, 
                        sum_or_max=True, remove_special_tokens=False):
  # Kudos to Marcos Piau
  with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=True):
    tokenized_text = tokenizer(text, max_length=max_length, truncation=True, 
                              return_tensors='pt', 
                              return_special_tokens_mask=True).to(device)
    model.to(device)
    model.eval()
    
    with torch.no_grad():
      output = model(tokenized_text['input_ids'].to(device), 
                     attention_mask=tokenized_text['attention_mask'].to(device))

    w_ij = output.logits[0,:]

    mask_tokens_validos = 1 - tokenized_text['special_tokens_mask'].to(device)

    #Kudos Leandro Carísio
    #1) Removes all dimenstions of size 1 (here, it's the first one)
    #2) Adds a 1 dimension to the end (equivalent to transpose, I think)
    #3) Replicates (expands) up to the vocabulary size -> 5 X 30k 
    if remove_special_tokens:
      mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
    else:
      mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
      mask = torch.ones(mask.size()).to(device)

    if sum_or_max:
      wj = torch.sum(torch.log(1 + relu(w_ij*mask)), dim=0)
    else:
      wj = torch.max(torch.log(1 + relu(w_ij*mask)), dim=0)[0]

  #According to ChatGPT: In PyTorch, the to_sparse() method is used to convert a 
  #dense tensor into a sparse tensor.  A dense tensor is a tensor that contains 
  #all elements, including those with zero values. In contrast, a sparse tensor 
  #is a tensor that only stores the non-zero elements, along with their indices. 
  #Sparse tensors are useful when working with large tensors with mostly zero 
  #values, as they can save memory and computational resources.
  return wj.to_sparse()


### Evaluating models vector sparsity

In [ ]:
text = "Where eat pizza"

In [ ]:
sparse_vec = vectorize_to_sparse(text, sum_or_max=False)
sparse_vec.indices().shape

torch.Size([1, 31])

In [ ]:
sparse_vec = vectorize_to_sparse(text, tokenizer_2, model_2, sum_or_max=False)
sparse_vec.indices().shape

torch.Size([1, 72])

In [ ]:
sparse_vec = vectorize_to_sparse(text, tokenizer_3, model_3, sum_or_max=False)
sparse_vec.indices().shape

torch.Size([1, 52])

In [ ]:
sparse_vec = vectorize_to_sparse(text, sum_or_max=False)
sparse_vec.indices().shape

indices = sparse_vec.indices()
tokenizer.convert_ids_to_tokens(indices[0])

['to',
 'are',
 'where',
 'later',
 'food',
 'guy',
 'places',
 'movie',
 'italy',
 'box',
 'mike',
 'joe',
 'store',
 'usa',
 'tony',
 'eat',
 'dinner',
 'restaurant',
 'alan',
 'cole',
 'dick',
 'bite',
 'pat',
 'package',
 'shelter',
 'cab',
 'foods',
 'pizza',
 'toby',
 'sandwiches',
 'domino']

## Evaluation on TREC-COVID

### Passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-04-23 03:03:17--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.35.7.13, 13.35.7.88, 13.35.7.102, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1682478198&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-8/trec-covid/corpus.jsonl already exists; do you wish to overwrite (y or n)? ^C


### Queries download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-04-23 03:03:46--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.35.7.13, 13.35.7.102, 13.35.7.28, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1682478226&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVz

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-8/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? ^C


### Queries vs passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-04-23 03:03:53--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 13.35.7.13, 13.35.7.102, 13.35.7.28, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.02s   

2023-04-23 03:03:53 (51.4 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

### Creating the queries sparse vectors

In [ ]:
model = model_3
tokenizer = tokenizer_3

In [ ]:
batch_size = 32

In [ ]:
query_ids = []
query_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)

test_queries_tokenized = tokenizer(query_texts, max_length=max_length, 
                                   truncation=True, padding=True, 
                                   return_special_tokens_mask=True)
dataset_queries_test = Dataset(test_queries_tokenized)
dataloader_queries_test = data.DataLoader(dataset_queries_test, 
                                          collate_fn=collate_fn,
                                          batch_size=batch_size, shuffle=False)

In [ ]:
len(dataset_queries_test)

50

In [ ]:
def vectorize_to_sparse_batch(batch, model=model_1, sum_or_max=True, 
                              remove_special_tokens=True):
  model.to(device)
  model.eval()
  
  with torch.no_grad():
    output = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))

  w_ij = output.logits.to(device)

  #Kudos Leandro Carísio
  mask_tokens_validos = batch['attention_mask'].to(device)
  mask = mask_tokens_validos.unsqueeze(-1).expand(w_ij.size())
  
  if sum_or_max:
    wj = torch.sum(torch.log(1 + relu(w_ij*mask)), dim=1)
  else:
    wj = torch.max(torch.log(1 + relu(w_ij*mask)), dim=1)[0]

  #According to ChatGPT: In PyTorch, the to_sparse() method is used to convert a 
  #dense tensor into a sparse tensor.  A dense tensor is a tensor that contains 
  #all elements, including those with zero values. In contrast, a sparse tensor 
  #is a tensor that only stores the non-zero elements, along with their indices. 
  #Sparse tensors are useful when working with large tensors with mostly zero 
  #values, as they can save memory and computational resources.
  return wj.to_sparse()

In [ ]:
from tqdm.auto import tqdm

queries_matrix = None

for query_batch in tqdm(dataloader_queries_test, mininterval=0.5, desc='Test', 
                                        disable=False):
  query_vector = vectorize_to_sparse_batch(query_batch, model=model, sum_or_max=False)

  if queries_matrix is None:
   queries_matrix = query_vector
  else:
   queries_matrix = torch.cat((queries_matrix, query_vector), dim=0)

Test:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
queries_matrix

tensor(indices=tensor([[    0,     0,     0,  ...,    49,    49,    49],
                       [ 1011,  1039,  2220,  ..., 28389, 28848, 28896]]),
       values=tensor([0.3317, 0.3782, 0.0831,  ..., 0.3011, 1.6011, 0.2566]),
       device='cuda:0', size=(50, 30522), nnz=2977, layout=torch.sparse_coo)

In [ ]:
indices = queries_matrix[1].coalesce().indices()
print(indices.shape)
tokenizer.convert_ids_to_tokens(indices[0])

torch.Size([1, 74])


['in',
 'they',
 'when',
 'during',
 'way',
 'how',
 'because',
 'body',
 'help',
 'change',
 'process',
 'changed',
 'movement',
 'relationship',
 'cause',
 'changes',
 'response',
 'effect',
 'winter',
 'spread',
 'transport',
 'heat',
 'function',
 'protection',
 'escape',
 'storm',
 'environment',
 'damage',
 'method',
 'formation',
 'impact',
 'disease',
 'wave',
 'environmental',
 'rain',
 'weather',
 'reaction',
 'signal',
 'flow',
 'temperature',
 'threat',
 'behavior',
 'crash',
 'responded',
 'factors',
 'transmission',
 'adaptation',
 'respond',
 'hurricane',
 'flood',
 'mechanism',
 'affect',
 'virus',
 'earthquake',
 'variation',
 'infection',
 'gravity',
 'gavin',
 'tracking',
 'thermal',
 'react',
 'pollution',
 'filter',
 'warming',
 '##cci',
 'immunity',
 '##vid',
 'leak',
 'contamination',
 'flu',
 'corona',
 '##virus',
 '##ergy',
 'pathogen']

### Creating the passages sparse vectors

In [ ]:
passage_ids = []
passage_texts = []
id_to_text = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_text[id] = text

#Sorts the passages by length
passage_indices = sorted(range(len(passage_texts)), 
                         key=lambda k: len(passage_texts[k]))
passage_texts = sorted(passage_texts, key=lambda k: len(k))
passage_ids = sorted(passage_ids, key=lambda k: len(id_to_text[k]))

In [ ]:
test_passages_tokenized = tokenizer(passage_texts, max_length=max_length, 
                                   truncation=True, padding=True, 
                                   return_special_tokens_mask=True)
dataset_passages_test = Dataset(test_passages_tokenized)
dataloader_passages_test = data.DataLoader(dataset_passages_test, 
                                          batch_size=batch_size, shuffle=False, 
                                          collate_fn=collate_fn)

In [ ]:
with open(f"{main_dir}/trec-covid/passage_ids.pickle", "wb") as f:
  pickle.dump(passage_ids, f)

In [ ]:
with open(f"{main_dir}/trec-covid/passage_texts.pickle", "wb") as f:
  pickle.dump(passage_texts, f)

In [ ]:
with open(f"{main_dir}/trec-covid/dataloader_passages_test.pickle", "wb") as f:
  pickle.dump(dataloader_passages_test, f)

In [ ]:
with open(f"{main_dir}/trec-covid/dataset_passages_test.pickle", "wb") as f:
  pickle.dump(dataset_passages_test, f)

In [ ]:
with open(f"{main_dir}/trec-covid/passage_ids.pickle", "rb") as f:
  passage_ids = pickle.load(f)

In [ ]:
with open(f"{main_dir}/trec-covid/passage_texts.pickle", "rb") as f:
  passage_texts = pickle.load(f)

In [ ]:
with open(f"{main_dir}/trec-covid/dataloader_passages_test.pickle", "rb") as f:
  dataloader_passages_test = pickle.load(f)

In [ ]:
with open(f"{main_dir}/trec-covid/dataset_passages_test.pickle", "rb") as f:
  dataset_passages_test = pickle.load(f)

In [ ]:
docs_matrix = None

# kudos Marcos Piau
with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
  for doc_batch in tqdm(dataloader_passages_test, mininterval=0.5, desc='Test', 
                                          disable=False):
    doc_vector = vectorize_to_sparse_batch(doc_batch, model=model, sum_or_max=False)

    if docs_matrix is None:
      docs_matrix = doc_vector
    else:
      docs_matrix = torch.cat((docs_matrix, doc_vector), dim=0)

Test:   0%|          | 0/5355 [00:00<?, ?it/s]

In [ ]:
with open(f"{main_dir}/trec-covid/passages_matrix.pickle", "wb") as f:
  pickle.dump(docs_matrix, f)

In [ ]:
with open(f"{main_dir}/trec-covid/passages_matrix.pickle", "rb") as f:
  passages_matrix = pickle.load(f)

In [ ]:
passages_matrix.shape

torch.Size([171332, 30522])

In [ ]:
indices = passages_matrix[134644].coalesce().indices()
indices.shape

torch.Size([1, 213])

In [ ]:
tokenizer.convert_ids_to_tokens(indices.squeeze())

['m',
 'n',
 'to',
 'for',
 'his',
 'time',
 'would',
 'over',
 'years',
 'used',
 'long',
 'group',
 'left',
 'good',
 '##m',
 'use',
 '##on',
 'help',
 'light',
 'control',
 'field',
 'role',
 '##f',
 'radio',
 'period',
 'common',
 'training',
 'week',
 'currently',
 'study',
 'arm',
 'performed',
 'studies',
 'model',
 'weeks',
 'active',
 'shoulder',
 'pain',
 'studied',
 'male',
 'neck',
 '##to',
 'activities',
 'results',
 'effect',
 'failed',
 'increase',
 'extended',
 'machine',
 'broken',
 'soft',
 'electric',
 'effects',
 'solo',
 'treatment',
 'tears',
 'activity',
 'damage',
 'shoulders',
 'ray',
 'joint',
 'analysis',
 'method',
 'assigned',
 '##me',
 '##it',
 '##ff',
 'fields',
 'models',
 'drug',
 'wave',
 'injury',
 'adult',
 'investigation',
 'da',
 'charlie',
 'failure',
 'functions',
 'device',
 'properties',
 'weapon',
 'shock',
 'injured',
 'improve',
 'techniques',
 '##ons',
 '##ue',
 'benefit',
 'treated',
 'bone',
 'bones',
 'surgery',
 'electrical',
 'techniqu

## Inverted Index

### Index building

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


Lucene's analyzer does not sufficiently remove stopwords. Therefore, NLTK's stopword list was additionally used.

In [ ]:
def preprocess_and_tokenize(text):
  tokenized_text = tokenizer(text, max_length=max_length, 
                                   truncation=True, padding=True, 
                                   return_special_tokens_mask=True)
  return tokenized_text["input_ids"], tokenized_text["special_tokens_mask"]

Next, we build the inverted index to relate the tokens to the documents where they occur.  

In [ ]:
!head {main_dir}/trec-covid/corpus.jsonl

{"_id": "ug7v899j", "title": "Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia", "text": "OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60

In [ ]:
passage_ids[134644]

'hqix7l5z'

In [ ]:
passage_texts[134644]

'Pulsed electromagnetic field therapy improves tendon-to-bone healing in a rat rotator cuff repair model. Rotator cuff tears are common musculoskeletal injuries often requiring surgical intervention with high failure rates. Currently, pulsed electromagnetic fields (PEMFs) are used for treatment of long-bone fracture and lumbar and cervical spine fusion surgery. Clinical studies examining the effects of PEMF on soft tissue healing show promising results. Therefore, we investigated the role of PEMF on rotator cuff healing using a rat rotator cuff repair model. We hypothesized that PEMF exposure following rotator cuff repair would improve tendon mechanical properties, tissue morphology, and alter in vivo joint function. Seventy adult male Sprague-Dawley rats were assigned to three groups: bilateral repair with PEMF (n = 30), bilateral repair followed by cage activity (n = 30), and uninjured control with cage activity (n = 10). Rats in the surgical groups were sacrificed at 4, 8, and 16 we

In [ ]:
main_dir

'/content/gdrive/MyDrive/Unicamp-aula-8'

In [ ]:
import array
import pandas as pd
from collections import defaultdict
from collections import Counter
import pickle
import os

index_path = f"{main_dir}/index.pickle"

def load_or_build_inverted_index():
  if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      index = pickle.load(f)
  else:
    print("Building inverted index...")
    inverted_index = dict()
    idx = 0

    def process(doc_id, idx):
      assert passage_ids[idx] == doc_id
      doc_vec = passages_matrix[idx]
      doc_vec = doc_vec.coalesce()
      indices = doc_vec.indices()[0]
      values = doc_vec.values()

      for token_id, wj in zip(indices, values):
        token_id = token_id.item()
        wj = wj.item()
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["docs"].append(idx)
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["wj"].append(wj)

    for i in tqdm(range(passages_matrix.shape[0])):
      process(passage_ids[i], i)
      
    index = {"inverted_index": inverted_index}

    with open(index_path, "wb") as f:
      pickle.dump(index, f)

  return index

In [ ]:
index = load_or_build_inverted_index()

Loading index...


In [ ]:
inverted_index = index["inverted_index"]

In [ ]:
len(inverted_index)

26050

### Searching

In [ ]:
import math
import torch.nn.functional as F
import time

def search_by_query_vector_in_inverted_index(query_vec, k):
  query_vec = query_vec.coalesce()
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  doc_ids = []
  indices = query_vec.indices()[0]
  values = query_vec.values()

  for token_id, wj in zip(indices, values):
    token_id = token_id.item()
    wj = wj.item()
    
    if token_id in inverted_index:
      doc_ids = inverted_index[token_id]["docs"]
      wjs = inverted_index[token_id]["wj"]

      for idx, doc_wj in zip(doc_ids, wjs):
        doc_scores[passage_ids[idx]] += wj * doc_wj
        
  doc_scores = dict(sorted(doc_scores.items(), key=lambda x:x[1], 
                           reverse=True)[:k])
          
  return doc_scores
     

In [ ]:
import math
import torch.nn.functional as F
import time

def search_by_query_vector_in_docs_matrix(query_vec, k):
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  scores = torch.sparse.mm(passages_matrix, query_vec.unsqueeze(-1))   
  indices = scores.indices()[0].cpu().numpy()
  values = scores.values().cpu().numpy()
  
  doc_scores = {passage_ids[indices[i]]:values[i] for i in range(len(indices))}  
  doc_scores = dict(sorted(doc_scores.items(), key=lambda x:x[1], 
                           reverse=True)[:k])
    
  return doc_scores
     

In [ ]:
%time doc_scores_inv_index = search_by_query_vector_in_inverted_index(queries_matrix[0], 1000)

CPU times: user 805 ms, sys: 12.6 ms, total: 818 ms
Wall time: 807 ms


In [ ]:
doc_scores_inv_index

{'1mjaycee': 27.96817747624557,
 'sr0umk33': 27.91484271668198,
 '3ll2tlzr': 27.897954601995686,
 'e6h1qvdk': 27.884275484351136,
 '958u08vb': 27.706056948305996,
 'dx3jywv7': 27.687253941749947,
 '4dtk1kyh': 27.653983679067217,
 'xuczplaf': 27.5936025466332,
 'utsr0zv7': 27.49429397418127,
 'sh7lrdou': 27.26857105784785,
 'jbtrdvhe': 27.123276345266053,
 'm4y8tf6u': 27.026586615581294,
 'deee71uw': 26.90269942219622,
 '6k5ac3f2': 26.683767451075937,
 'n9bn0ag7': 26.67271044966032,
 '1c47w4q5': 26.594125708193943,
 'eju7wnb9': 26.388904532722044,
 'syhgjmf8': 26.282492706262772,
 'wco27nop': 26.249053986761748,
 'nd5xpk0t': 26.19080653431715,
 'fye4fpj2': 26.18431724567127,
 'y3uzb4dx': 26.177154593234977,
 'oszhjnd1': 26.17471388273018,
 '75773gwg': 26.142174978035968,
 'iohvj16d': 26.139054953093567,
 '8bohkzfq': 26.097695547418407,
 'fv355ywl': 26.078381942486203,
 'm1evoj5p': 26.02912989825666,
 'cjrji0ev': 26.017963028959255,
 'o8towjim': 25.933637784658618,
 'dqxfcwyu': 25.919989

In [ ]:
%time doc_scores_matrix = search_by_query_vector_in_docs_matrix(queries_matrix[0],1000)

CPU times: user 375 ms, sys: 3.2 ms, total: 378 ms
Wall time: 393 ms


In [ ]:
doc_scores_matrix

{'1mjaycee': 27.968176,
 'sr0umk33': 27.914843,
 '3ll2tlzr': 27.897953,
 'e6h1qvdk': 27.884275,
 '958u08vb': 27.706059,
 'dx3jywv7': 27.687256,
 '4dtk1kyh': 27.653984,
 'xuczplaf': 27.593601,
 'utsr0zv7': 27.494293,
 'sh7lrdou': 27.268572,
 'jbtrdvhe': 27.123276,
 'm4y8tf6u': 27.026587,
 'deee71uw': 26.902699,
 '6k5ac3f2': 26.683767,
 'n9bn0ag7': 26.67271,
 '1c47w4q5': 26.594126,
 'eju7wnb9': 26.388906,
 'syhgjmf8': 26.282494,
 'wco27nop': 26.249056,
 'nd5xpk0t': 26.190807,
 'fye4fpj2': 26.184317,
 'y3uzb4dx': 26.177155,
 'oszhjnd1': 26.174713,
 '75773gwg': 26.142174,
 'iohvj16d': 26.139055,
 '8bohkzfq': 26.097694,
 'fv355ywl': 26.07838,
 'm1evoj5p': 26.02913,
 'cjrji0ev': 26.017963,
 'o8towjim': 25.933636,
 'dqxfcwyu': 25.919987,
 'l0sbncnp': 25.839329,
 'v9ftmdlv': 25.705902,
 'aura0th0': 25.679142,
 '4zb63c0p': 25.678915,
 'vx7ebtbp': 25.677608,
 'rtw8m00s': 25.63451,
 'uvoavj3z': 25.624748,
 'p3681yit': 25.592524,
 'r2ynbnxx': 25.569317,
 'ptizke03': 25.518328,
 'k1tuda5o': 25.4844

In [ ]:
doc_scores_1['l4o7nicc']

20.751570096582036

In [ ]:
doc_scores_2['l4o7nicc']

20.75157

In [ ]:
k = 1000

In [ ]:

from collections import defaultdict
import time

run_splade_inv = defaultdict(list)

start = time.time()

for i in range(queries_matrix.shape[0]):
  doc_scores = search_by_query_vector_in_inverted_index(queries_matrix[i], k)
  query_id = query_ids[i]
  run_splade_inv["query"] += [query_id] * k
  run_splade_inv["docid"] += doc_scores.keys()
  run_splade_inv["score"] += doc_scores.values()
  run_splade_inv["q0"] += ["q0"] * k
  run_splade_inv["rank"] += list(range(1,k+1))
  run_splade_inv["system"] += ["splade"] * k

end = time.time()
print("Time spent = ", end - start)

Time spent =  42.47600269317627


In [ ]:

from collections import defaultdict

run_splade_matrix = defaultdict(list)

start = time.time()

for i in range(queries_matrix.shape[0]):
  doc_scores = search_by_query_vector_in_docs_matrix(queries_matrix[i], k)
  query_id = query_ids[i]
  run_splade_matrix["query"] += [query_id] * k
  run_splade_matrix["docid"] += doc_scores.keys()
  run_splade_matrix["score"] += doc_scores.values()
  run_splade_matrix["q0"] += ["q0"] * k
  run_splade_matrix["rank"] += list(range(1,k+1))
  run_splade_matrix["system"] += ["splade"] * k

end = time.time()
print("Time spent = ", end - start)

Time spent =  17.441141605377197


### Evaluation

In [ ]:
import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [ ]:
from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10']    

In [ ]:
eval_ndcg10(run_splade_inv)

0.7269462314492029

In [ ]:
eval_ndcg10(run_splade_matrix)

0.7269462314492029